# 1. Web Scraping

In [2]:
url = "https://www.aljazeera.com/where/mozambique/"

Its a dynamic website so I use selenium to get the content

In [168]:
!pip install html5print
!pip install flair
!pip install -U kaleido
!pip install selenium
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 79.9 MB 1.3 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached urllib3-1.26.9-py2.py3-none-any.whl (138 kB)
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.25.11
    Uninstalling urllib3-1.25.11:
      Successfully uninstalled urllib3-1.25.11
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.27.1 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you 

Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:6 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:7 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:14 http://archive.ubuntu.com/ubuntu b

In [7]:
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

In [118]:
from selenium import webdriver
from bs4 import BeautifulSoup

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
wd = webdriver.Chrome('chromedriver',options=chrome_options)

In [143]:
def get_info_from_div(div_soup):
  article_info = {}

  # 1. Get title
  article_info["title"] = div_soup.find('h3', class_="gc__title").a.span.text.replace('\xad', '')
    
  # 2. Get summery
  article_info["excerpt"] = div_soup.find('div', class_="gc__excerpt").p.text.replace('\xad', '')

  # 3. Get image url
  article_info["img"] = "https://www.aljazeera.com" + div_soup.find('img')['src']

  # 4. Get publish time
  article_info["publish_date"] = div_soup.find('div', class_="gc__date__date").find_all('span')[-1].text.replace('\xad', '')

  # 5. Get article url
  article_info["article"] = "https://www.aljazeera.com" + div_soup.find('h3', class_="gc__title").a['href']

  return article_info

wd.get(url)

source = wd.page_source
soup = BeautifulSoup(source, features='lxml')

recent_ten_articles = []

recent_four_articles = soup.find('ul', class_="featured-articles-list")

if recent_four_articles:
  for recent_four_article_soup in recent_four_articles.find_all('article'):
    recent_ten_articles.append(get_info_from_div(recent_four_article_soup))

recent_articles = soup.find('section', id="news-feed-container")
if recent_articles:
  for recent_article_soup in recent_articles.find_all('article'):
    recent_ten_articles.append(get_info_from_div(recent_article_soup))


In [144]:
from flair.models import TextClassifier
from flair.data import Sentence
sia = TextClassifier.load('en-sentiment')
def flair_prediction(x):
    sentence = Sentence(x)
    sia.predict(sentence)
    score = sentence.labels[0]

    print(score)
    if "POSITIVE" in str(score):
      score_string = str(score).split('POSITIVE')[-1].replace('(','').replace(')','')
      print(score_string)
      return "pos", float(score_string)
    elif "NEGATIVE" in str(score):
      score_string = str(score).split('NEGATIVE')[-1].replace('(','').replace(')','')
      print(score_string)
      return "neg", float(score_string)
    else:
        return "neu", 0

2022-06-09 05:13:08,342 loading file /root/.flair/models/sentiment-en-mix-distillbert_4.pt


In [145]:
from tqdm import tqdm

final_article_results = []
for recent_ten_article in tqdm(recent_ten_articles):
  wd.get(recent_ten_article['article'])
  article_soup = BeautifulSoup(wd.page_source, features='lxml') 

  article_content_soup = article_soup.find('div', class_="wysiwyg wysiwyg--all-content css-1ck9wyi")
  full_content = ""
  if article_content_soup:
    for paragraph in article_content_soup.findChildren('p',recursive=False):
      full_content+=paragraph.text+'\n'
    recent_ten_article['full_content'] = full_content
    recent_ten_article["sentiment"], recent_ten_article["score"]  = flair_prediction(recent_ten_article["full_content"])

     

    final_article_results.append(recent_ten_article)
  

  7%|▋         | 1/14 [00:03<00:47,  3.64s/it]

Sentence: "Heavy rains in South Africa have forced hundreds of people to flee their homes on the east coast , according to officials , only a month after the worst floods in living memory killed more than 400 . The floods over the weekend hit infrastructure , roads , bridges and damaged buildings , mainly in the port city of Durban , the capital of KwaZulu-Natal province and home to some 3.9 million people . “ Some areas are inaccessible and have become islands at this stage ,” KwaZulu-Natal Premier Sihle Zikalala said during a news conference on Sunday . No fatalities were immediately recorded , but Zikalala added : “ We are yet to receive a full comprehensive report on the impact of these latest heavy rains , but as reports come in a worrying picture is however emerging . ” In April , torrential rain led to floods , landslides and the death of 435 people in KwaZulu-Natal and authorities said survivors were left without drinking water for nearly two weeks . Repairs are still ongoing a

 14%|█▍        | 2/14 [00:07<00:42,  3.52s/it]

Sentence: "Tropical Cyclone Gombe has killed at least 53 people since it hit Mozambique a week ago , a sharp rise from earlier estimates . According to the National Institute of Disaster Management ( INGC ) on Thursday , another 80 people have been injured and 400,000 affected since the cyclone swept into northern and central areas of the country , flooding towns and destroying houses . The initial death toll in the southern African country was estimated at seven . Data showed Gombe had completely destroyed 45,000 homes and partially destroyed 30,000 . In the past few years , Southern Africa has suffered repeated devastating cyclones of the type that used to be relatively rare , and scientists believe climate change is fuelling their intensity via warming of the Indian Ocean . In January , Tropical Storm Ana killed at least 88 people across Southern and Eastern Africa ." → NEGATIVE (0.996)
 0.996


 21%|██▏       | 3/14 [00:09<00:32,  2.98s/it]

...

Sentence: "As we entered 2021 , there were 82.4 million people around the world displaced by conflict or persecution . Thirty million of those are refugees , the rest are displaced within their own country ( 48 million ) or asylum-seekers ( 4.1 million ) , according to UNHCR ’s latest report . Nearly half of these forcibly displaced people are children . 55 percent of refugees come from three countries : Syria , Palestine and Venezuela . During the COVID-19 pandemic , “ everything else has stopped , including the economies , but wars and conflict and violence and discrimination and persecution , all the factors that pushed these people to flee , have continued ,” UNHCR chief Filippo Grandi said . In 2020 , 1.27 million people from 64 countries became refugees . The infographic below shows the desperate journeys these people took despite the additional challenges brought on by COVID-19 . Africa accounts for more than one-third of the world ’s displaced people . By the end of 2020 , at l

 93%|█████████▎| 13/14 [00:57<00:05,  5.03s/it]

Sentence: "Fighters in conflict-hit northern Mozambique abducted dozens of children during raids in 2020 , according to a new analysis by Save the Children . The charity said in a report on Wednesday the “ abduction of children has become a new and alarmingly regular tactic by armed groups ” in Cabo Delgado province , where worsening fighting over the past three-and-a-half years has killed nearly 3,000 people and displaced more than 700,000 , half of whom are children . Save the Children said “ at least 51 children , most of them girls ” were seized by non-state armed groups in the region last year , adding that the numbers involved were likely “ far higher ” than its estimates , which were based on data collected by the Armed Conflict Location & Event Data Project and reflected only reported cases . It warned that the victims are at risk of sexual violence , early marriage and being used as fighters in the conflict . “ Being abducted , witnessing abductions , experiencing attacks , be

100%|██████████| 14/14 [01:02<00:00,  4.47s/it]

Sentence: "White contractors were prioritised for evacuation ahead of Black locals during a rescue operation following an attack in March by an armed group known locally as al-Shabab on the Mozambican town of Palma , according to Amnesty International . In a report compiled from interviews with 11 Black survivors , Amnesty on Thursday charged that even dogs were pulled to safety before Black people by a helicopter that airlifted civilians from a hotel where they had sought refuge . “ White contractors [ were ] airlifted to safety before local Black people ,” said Amnesty , adding the hotel manager took his two German Shepherd dogs on the rescue helicopter , leaving people behind . Dyck Advisory Group ( DAG ) , a South Africa private company hired to help the Mozambique government fight the armed group and which was involved in the rescue operation , rejected the allegations . “ We rescued 24 people from the Amarula Lodge on the 25 March 2021 this consisted of 6 white persons and 18 Bla

In [152]:
import json
with open('data.json', 'w') as f:
    json.dump(final_article_results, f, ensure_ascii=False)

with open('data.json', 'r') as f:
    data_from_json = json.load(f)

In [164]:
score_list = []
date_list = []
for article in data_from_json:
  print(article['article'])
  print(article["title"])
  print(article["sentiment"], article["score"])
  print(article['publish_date'])
  print()
  if article["sentiment"] == "neg":
    score_list.append(0-article["score"])
  else:
    score_list.append(article["score"])
  date_list.append(article['publish_date'])

https://www.aljazeera.com/news/2022/5/23/floods-hit-south-africas-kwazulu-natal-province-again
Floods hit South Africa’s KwaZulu-Natal province again
neg 0.9898
23 May 2022

https://www.aljazeera.com/news/2022/3/18/mozambique-cyclone-gombe-death-toll-rises-to-53
Mozambique: Cyclone Gombe death toll rises to 53
neg 0.996
18 Mar 2022

https://www.aljazeera.com/news/2022/3/4/mozambique-announces-new-prime-minister-and-finance-minister
Mozambique announces new prime minister after cabinet reshuffle
neg 0.9038
4 Mar 2022

https://www.aljazeera.com/economy/2022/3/1/analysis-can-african-gas-replace-russian-supplies-to-europe
Analysis: Can African gas replace Russian supplies to Europe?
neg 0.9908
1 Mar 2022

https://www.aljazeera.com/news/2022/1/27/at-least-70-dead-from-tropical-storm-ana-in-southern-africa
Dozens dead from Tropical Storm Ana in southern Africa
neg 0.9945
27 Jan 2022

https://www.aljazeera.com/news/2022/1/12/southern-africa-bloc-sadc-extends-mozambique-mission
Southern Africa

In [165]:
print(score_list)
print(date_list)
date_list.reverse()
score_list.reverse()

[-0.9898, -0.996, -0.9038, -0.9908, -0.9945, 0.6893, -0.9069, -0.7817, -0.9964, -0.9748, -0.9716, -0.9977, -0.9997]
['23 May 2022', '18 Mar 2022', '4 Mar 2022', '1 Mar 2022', '27 Jan 2022', '12 Jan 2022', '24 Sep 2021', '8 Aug 2021', '10 Jul 2021', '23 Jun 2021', '20 Jun 2021', '9 Jun 2021', '14 May 2021']


In [169]:
import plotly.graph_objects as go

fig = go.Figure([go.Scatter(x=date_list, y=score_list)])
fig.show()


ValueError: ignored